In [1]:
#importing necessary libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder


In [2]:
# importing and separating data
all_data = pd.read_csv('all_data.csv')

all_data = all_data.dropna()
# making X data and Y data
X = all_data.drop(['justicesDecision', 'caseId', 'term', 'dateDecision', 'fullJusticeName', 'majority', 'justiceName'], axis=1)
y = all_data['justicesDecision']



,decisionType,naturalCourt,chief,petitioner,respondent,caseSource,certReason,lcDispositionDirection,declarationUncon,caseDisposition,...,decisionDirectionDissent,authorityDecision1,authorityDecision2,lawType,lawSupp,justice,vote,direction,presAppointed,presAffiliation
18,1,1709,Roberts,28.0,333.0,32.0,12.0,2.0,1.0,0.0,...,0.0,4.0,5.0,3.0,304.0,117,1.0,1.0,Donald Trump,1
59,1,1709,Roberts,172.0,172.0,8.0,10.0,1.0,1.0,0.0,...,0.0,7.0,4.0,6.0,600.0,117,2.0,1.0,Donald Trump,1
101,1,1709,Roberts,119.0,237.0,22.0,12.0,2.0,1.0,0.0,...,0.0,7.0,4.0,3.0,365.0,117,1.0,1.0,Donald Trump,1
110,1,1709,Roberts,122.0,134.0,29.0,12.0,2.0,1.0,0.0,...,0.0,1.0,7.0,1.0,139.0,117,1.0,1.0,Donald Trump,1
116,1,1709,Roberts,7.0,165.0,25.0,11.0,2.0,1.0,0.0,...,0.0,2.0,7.0,2.0,230.0,117,1.0,1.0,Donald Trump,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81991,1,1405,Warren,382.0,151.0,25.0,2.0,2.0,1.0,0.0,...,0.0,5.0,4.0,3.0,356.0,81,1.0,1.0,Franklin Roosevelt,0
81994,1,1405,Warren,249.0,151.0,25.0,3.0,1.0,1.0,0.0,...,0.0,4.0,3.0,3.0,350.0,81,1.0,2.0,Franklin Roosevelt,0
81996,1,1409,Warren,362.0,184.0,22.0,12.0,2.0,1.0,0.0,...,0.0,5.0,4.0,3.0,332.0,81,1.0,2.0,Franklin Roosevelt,0
82003,1,1408,Warren,382.0,141.0,29.0,12.0,1.0,1.0,0.0,...,0.0,4.0,5.0,3.0,356.0,81,1.0,2.0,Franklin Roosevelt,0


In [3]:
# have to do preprocessing and one hot encode since the variables are categorical
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_encoded = encoder.fit_transform(X)



In [4]:
# then I split data 
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.1, random_state=42)

In [5]:
# now I train random forest classifier
randfor = RandomForestClassifier(n_estimators=10, class_weight = 'balanced', random_state=42) # n_estimators means the amount of trees 

randfor.fit(X_train, y_train) #training the model 


RandomForestClassifier(class_weight='balanced', n_estimators=10,
                       random_state=42)

In [6]:
# now I a making predictions and classifying the model 

predictions = randfor.predict(X_test)

# accuracy calculation
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy}')

Accuracy: 1.0


In [7]:
importances = randfor.feature_importances_ #feature importances 


# now relating the encoded features back to the acutal ones
feature_names = encoder.get_feature_names_out(X.columns)
original_feature_importance = dict.fromkeys(X.columns, 0)

for encoded_feature, importance in zip(feature_names, importances):
    original_feature = encoded_feature.split('_')[0]  # using default one hot encoding feature naming
    original_feature_importance[original_feature] += importance

sorted_by_importance = sorted(original_feature_importance.items(), key=lambda x: x[1], reverse=True)

# printing the sorted features now and their importance
for feature, importance in sorted_by_importance:
    print(f"{feature}: {importance}")


vote: 0.7944914537498537
direction: 0.046953204900925895
justice: 0.022970845439067772
decisionDirection: 0.01648256433818826
issue: 0.015428582967416594
petitioner: 0.01318121674181708
respondent: 0.013044761075705897
presAppointed: 0.011869618343209678
naturalCourt: 0.00850924825668201
caseSource: 0.00841249458246476
lawSupp: 0.007551103866995853
issueArea: 0.005271931176556722
certReason: 0.005179489971313941
authorityDecision2: 0.004811203766200513
lcDispositionDirection: 0.0046312291982807405
authorityDecision1: 0.004052724297579653
decisionType: 0.003743605386865106
partyWinning: 0.0036023875755988068
lawType: 0.0031099020018790407
presAffiliation: 0.002821928857703859
chief: 0.0027927628866159476
declarationUncon: 0.0006430250471273967
precedentAlteration: 0.00025956378507592043
decisionDirectionDissent: 0.00018515178687493223
caseDisposition: 0.0
